<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#MG-sample-subset-(n=100)" data-toc-modified-id="MG-sample-subset-(n=100)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>MG sample subset (n=100)</a></span><ul class="toc-item"><li><span><a href="#Writing-samples-file" data-toc-modified-id="Writing-samples-file-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Writing samples file</a></span></li><li><span><a href="#LLMGQC" data-toc-modified-id="LLMGQC-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>LLMGQC</a></span></li><li><span><a href="#Determining-which-worked" data-toc-modified-id="Determining-which-worked-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Determining which worked</a></span></li><li><span><a href="#LLMGQC" data-toc-modified-id="LLMGQC-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>LLMGQC</a></span></li><li><span><a href="#Determining-which-worked" data-toc-modified-id="Determining-which-worked-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Determining which worked</a></span></li><li><span><a href="#LLMGQC" data-toc-modified-id="LLMGQC-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>LLMGQC</a></span><ul class="toc-item"><li><span><a href="#Creating-genome-sample-map-table" data-toc-modified-id="Creating-genome-sample-map-table-5.6.1"><span class="toc-item-num">5.6.1&nbsp;&nbsp;</span>Creating genome-sample map table</a></span></li><li><span><a href="#Writing-table" data-toc-modified-id="Writing-table-5.6.2"><span class="toc-item-num">5.6.2&nbsp;&nbsp;</span>Writing table</a></span></li><li><span><a href="#Creating-features" data-toc-modified-id="Creating-features-5.6.3"><span class="toc-item-num">5.6.3&nbsp;&nbsp;</span>Creating features</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>Summary</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* create feature tables for [AlmeidaA et al., 2019 dataset](https://doi.org/10.1038/s41586-019-0965-1)

# Var

In [1]:
work_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/'

# checkM results for all MAGs
checkm_res_file = file.path(work_dir, 'mags-gut_qs50_checkm.tab')

# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
set.seed(18734)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



# Load

In [3]:
checkm_res = read.delim(checkm_res_file, sep='\t') 
checkm_res %>% nrow %>% print
checkm_res %>% head(n=3)

[1] 92143


MAG,Completeness,Contamination,Strain_heterogeneity,CheckM_lineage
SRR3496379_bin.19,96.98,0.67,0,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales
SRR3496379_bin.31,97.09,1.80,0,k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Prevotellaceae
SRR3496379_bin.37,78.63,0.17,100,k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales


# MG sample subset (n=100)

In [4]:
# getting NCBI accessions
checkm_res = checkm_res %>%
    mutate(acc = gsub('_bin\\.[0-9]+$', '', MAG)) 

checkm_res$acc %>% unique %>% length %>% print

[1] 10902


In [5]:
# randomly selecting subset of samples
acc_sub1 = sample(checkm_res$acc, 100)
acc_sub1 %>% length %>% print
acc_sub1 %>% sort %>% head

[1] 100


[1] "DRR042488"  "DRR042592"  "ERR011257"  "ERR1018311" "ERR1190619"
[6] "ERR1293605"

In [6]:
# number of MAGs corresponding to samples
checkm_res_f = checkm_res %>%
    filter(acc %in% acc_sub1) 

checkm_res_f %>% nrow

[1] 1782

In [7]:
# checkm stats
checkm_res %>%
    dplyr::select(Completeness, Contamination, Strain_heterogeneity) %>%
    summary

checkm_res_f %>%
    dplyr::select(Completeness, Contamination, Strain_heterogeneity) %>%
    summary

  Completeness    Contamination   Strain_heterogeneity
 Min.   : 50.03   Min.   :0.000   Min.   :  0.00      
 1st Qu.: 75.44   1st Qu.:0.220   1st Qu.:  0.00      
 Median : 88.77   Median :1.070   Median :  0.00      
 Mean   : 84.34   Mean   :1.563   Mean   : 14.67      
 3rd Qu.: 95.33   3rd Qu.:2.260   3rd Qu.: 20.00      
 Max.   :100.00   Max.   :9.730   Max.   :100.00      

  Completeness    Contamination   Strain_heterogeneity
 Min.   : 50.04   Min.   :0.000   Min.   :  0.00      
 1st Qu.: 76.49   1st Qu.:0.340   1st Qu.:  0.00      
 Median : 88.22   Median :1.200   Median :  0.00      
 Mean   : 84.37   Mean   :1.723   Mean   : 13.42      
 3rd Qu.: 94.67   3rd Qu.:2.510   3rd Qu.: 16.67      
 Max.   :100.00   Max.   :9.430   Max.   :100.00      

## Writing samples file

In [ ]:
samples_file = file.path(work_dir, 'samples_n100.txt')
checkm_res_f %>%
    rename('Sample' = acc) %>%
    mutate(Remote = Sample) %>%
    distinct(Sample, Remote) %>%
    mutate(Run = 1, Lane = 1) %>%
    write.table(file=samples_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', samples_file, '\n')

## LLMGQC

* downloading and QC of selected MG samples

```{bash}
(snakemake_dev) @ rick:/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/llmgqc
$ screen -L -S llmgqc-DM ./snakemake_sge.sh /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/config.yaml cluster.json /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/SGE_log 20
```

## Determining which worked

In [ ]:
p = file.path(work_dir, 'LLMGQC', 'final')
samps_success = list.files(P, 'R2_final.fq.gz', recursive=TRUE) %>% dirname
samps_success %>% length %>% print
samps_success %>% head

In [ ]:
# randomly selecting subset of new samples
acc_sub2 = sample(checkm_res$acc %>% unique, 100)
acc_sub2 %>% length %>% print
acc_sub2 %>% sort %>% head

In [ ]:
# number of MAGs corresponding to samples
checkm_res_f = checkm_res %>%
    filter(acc %in% acc_sub2) 

checkm_res_f %>% nrow

In [ ]:
# checking for overlap 
intersect(samps_success, checkm_res_f$acc) %>% print

checkm_res_f = checkm_res_f %>%
    filter(! acc %in% samps_success)

checkm_res_f$acc %>% unique %>% length %>% print
checkm_res_f %>% nrow %>% print

In [ ]:
samples_file = file.path(work_dir, 'samples_n99.txt')
checkm_res_f %>%
    rename('Sample' = acc) %>%
    mutate(Remote = Sample) %>%
    distinct(Sample, Remote) %>%
    mutate(Run = 1, Lane = 1) %>%
    write.table(file=samples_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', samples_file, '\n')

## LLMGQC

```{bash}
(snakemake_dev) @ rick:/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/llmgqc
$ screen -L -S llmgqc-DM ./snakemake_sge.sh /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/config.yaml cluster.json /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/SGE_log 20
```

## Determining which worked

In [ ]:
p = file.path(work_dir, 'LLMGQC', 'final')
samps_success = list.files(P, 'R2_final.fq.gz', recursive=TRUE) %>% dirname
samps_success %>% length %>% print
samps_success %>% head

In [ ]:
# subsampling
checkm_res_f = checkm_res %>%
    filter(acc %in% samps_success)
checkm_res_f$acc %>% unique %>% length %>% print
checkm_res_f %>% nrow %>% print

In [ ]:
# number of associated MAGs
cat('Number of MAGs:', checkm_res_f$MAG %>% unique %>% length, '\n')

In [ ]:
samples_file = file.path(work_dir, 'samples_n143.txt')
checkm_res_f %>%
    rename('Sample' = acc) %>%
    mutate(Remote = Sample) %>%
    distinct(Sample, Remote) %>%
    mutate(Run = 1, Lane = 1) %>%
    write.table(file=samples_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', samples_file, '\n')

## LLMGQC

```{bash}
(snakemake_dev) @ rick:/ebio/abt3_projects/databases_no-backup/DeepMAsED/bin/llmgqc
$ screen -L -S llmgqc-DM ./snakemake_sge.sh /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/config.yaml cluster.json /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/SGE_log 20
```

### Creating genome-sample map table

* for mapping metagenome reads to MAGs

In [ ]:
mag_dir = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/mags/'

samp_mag_idx = checkm_res_f %>%
    mutate(Fasta = sapply(as.character(MAG), function(x) paste(c(mag_dir, x, '.fa.gz'), collapse=''))) %>%
    rename('Taxon' = MAG,
           'Sample' = acc)

samp_mag_idx %>% nrow %>% print
samp_mag_idx %>% head(n=3)

In [ ]:
# getting read locations for all samples
P = '/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/LLMGQC/final/'
R1_files = list.files(P, 'R1_final.fq.gz', recursive=TRUE, full.names=TRUE) 
R2_files = list.files(P, 'R2_final.fq.gz', recursive=TRUE, full.names=TRUE) 

R1_files %>% length %>% print
R1_files %>% head(n=3)

R2_files %>% length %>% print
R2_files %>% head(n=3)

## joining into a df
samp_files = data.frame(Sample = basename(dirname(R1_files)),
                        Read1 = R1_files,
                        Read2 = R2_files)
samp_files %>% nrow %>% print
samp_files %>% head(n=3)

In [ ]:
# joining with samp-mag index
samp_mag_idx = samp_mag_idx %>%
    left_join(samp_files, c('Sample')) 

samp_mag_idx %>% nrow %>% print
samp_mag_idx %>% head(n=3)

In [ ]:
samp_mag_idx %>% 
    summary

### Writing table

In [11]:
samp_mag_idx_file = file.path(work_dir, 'samples_n143_w-MAGs.txt')
write.table(samp_mag_idx, samp_mag_idx_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', samp_mag_idx_file, '\n')

File written: /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019//samples_n143_w-MAGs.txt 


### Creating features

* Using DeepMAsED

In [8]:
work_dir1 = file.path(work_dir, 'samples_n143_MAG-feats')
work_dir1

[1] "/ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019//samples_n143_MAG-feats"

```{bash}
(snakemake) @ rick:/ebio/abt3_projects/software/dev/DeepMAsED/DeepMAsED-SM
$ screen -L -S DM-MAG ./snakemake_sge.sh /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/samples_n143_MAG-feats/config.yaml cluster.json /ebio/abt3_projects/databases_no-backup/DeepMAsED/MAG_datasets/AlmeidaA-2019/samples_n143_MAG-feats/SGE_log/ 24
[detached from 30285.DM-MAG]
```

## Summary

In [9]:
P = file.path(work_dir1, 'map')
feat_files = list.files(P, 'features.tsv.gz', recursive=TRUE)
feat_files %>% length

[1] 1519

# sessionInfo

In [10]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.2 LTS

Matrix products: default
BLAS: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRblas.so
LAPACK: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] bindrcpp_0.2.2 ggplot2_3.0.0  tidyr_0.8.1    dplyr_0.7.6   

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.17     bindr_0.1.1      magrittr_1.5     munsell_0.5.0   
 [5] tidyselect_0.2